In [227]:
import pandas as pd
import openpyxl as ox
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
os.getcwd()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'f:\\01PHD\\016Heat_Pump_Research\\01SN_Rproject'

In [228]:
df_permit = pd.read_csv('01Data/Trade_Permits_20251003.csv')
## 


In [229]:
df_final2= pd.read_excel('02ProcessedData/apt_trade_HP_permit_details_1028.xlsx')

## remove rows where StatusCurrent is 'Canceled' or 'Withdrawn'
df_final2 = df_final2[~df_final2['StatusCurrent'].isin(['Canceled', 'Withdrawn'])]
df_apt_unique_address = pd.read_excel('02ProcessedData/data_apt_unique_address.xlsx')
df_permit_parsed= pd.read_excel('02ProcessedData/parsed_permit_details_new_method.xlsx')
##join df_final2 with df_permit_parsed on permit_id to get the final dataframe with parsed permit details

df_final3 = df_final2.merge(df_permit_parsed, left_on='Permit Number', right_on='permit_id', how='left', suffixes=('', '_y'))
df_final3.info()

## dfinal3 summary by addr_clean_y and sum of tonnage_total and compressors_final
# df_final3_summary = df_final3.groupby('addr_clean_y').agg({'tonnage_total': 'sum', 'compressors_final': 'sum', 'Units_y':'mean', 'Existing Building Square Feet': 'mean', 'Stories_y': 'mean'}).reset_index()
# df_final3_summary['tonnage_cover_ratio'] = df_final3_summary['tonnage_total']* 700 / (df_final3_summary['Existing Building Square Feet'] )
# df_final3_summary['compressors_per_units'] = df_final3_summary['compressors_final'] / df_final3_summary['Units_y']

####### for df_final3, do a running total of tonnage_total and compressors_final and number_final by addr_clean_y and Year Completed,sorted by Year Completed
df_final3_summary = df_final3
df_final3_summary = df_final3_summary.sort_values(by=['addr_clean_y', 'Year Completed'])
df_final3_summary['tonnage_total_cumsum'] = df_final3_summary.groupby('addr_clean_y')['tonnage_total'].cumsum()
df_final3_summary['compressors_final_cumsum'] = df_final3_summary.groupby('addr_clean_y')['compressors_final'].cumsum()
# df_final3_summary['number_final_cumsum'] = df_final3_summary.groupby('addr_clean_y')['number_final'].cumsum()

df_final3_summary['tonnage_cover_ratio'] = df_final3_summary['tonnage_total_cumsum']* 700 / (df_final3_summary['Existing Building Square Feet'] )
df_final3_summary['tonnage_per_units'] = df_final3_summary['tonnage_total_cumsum'] / (df_final3_summary['Units'] )
df_final3_summary['compressors_per_units'] = df_final3_summary['compressors_final_cumsum'] / df_final3_summary['Units']
# df_final3_summary['number_final_unit_ratio'] = df_final3_summary['number_final_cumsum'] / df_final3_summary['Units_y']

## merge df_final3_summary with df_apt_unique_address on addr_clean_y and addr_clean
df_final3_summary = df_final3_summary.merge(df_apt_unique_address, left_on='addr_clean_y', right_on='addr_clean', how='left', suffixes=('', '_y'))
# df_final3_summary.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025_summary.xlsx', index=False)
df_final3_summary.describe().T



df_final3.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025.xlsx', index=False)



##merge df_final3_summary with df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 96 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Permit Number                    920 non-null    object 
 1   Permit_Content                   920 non-null    object 
 2   addr_clean_x                     920 non-null    object 
 3   BuildingName                     909 non-null    object 
 4   StarRating                       920 non-null    int64  
 5   Submarket                        920 non-null    object 
 6   City                             920 non-null    object 
 7   ZIPCode                          920 non-null    int64  
 8   Stories                          919 non-null    float64
 9   YearBuilt                        917 non-null    float64
 10  YearRenov                        38 non-null     float64
 11  Class                            920 non-null    object 
 12  TrueOwner             

,count,mean,std,min,25%,50%,75%,max
StarRating,921.0,3.838219e+00,5.180384e-01,2.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
ZIPCode,921.0,9.811376e+04,1.148418e+01,9.810100e+04,9.810400e+04,9.811200e+04,9.812200e+04,9.819900e+04
Stories,920.0,7.702174e+00,6.491176e+00,1.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,4.400000e+01
YearBuilt,918.0,2.007759e+03,2.310461e+01,1.900000e+03,2.009000e+03,2.015000e+03,2.019000e+03,2.023000e+03
YearRenov,38.0,2.010500e+03,9.802785e+00,1.978000e+03,2.006000e+03,2.015000e+03,2.016000e+03,2.019000e+03
Units,921.0,1.515114e+02,1.202085e+02,1.500000e+01,6.000000e+01,1.080000e+02,2.340000e+02,7.610000e+02
latitude,921.0,4.762592e+01,4.058561e-02,4.752012e+01,4.760993e+01,4.761877e+01,4.765652e+01,4.773255e+01
longitude,921.0,-1.223315e+02,2.616882e-02,-1.223889e+02,-1.223473e+02,-1.223276e+02,-1.223155e+02,-1.222617e+02
LEED,139.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
ParkingSpaces,485.0,1.436333e+02,1.327370e+02,0.000000e+00,4.100000e+01,1.120000e+02,2.000000e+02,9.160000e+02


## HP Decision Rule for having HP or Not


In [230]:
pattern = r'(?!.*(AC|air condition|a/c(?!.*(HVAC|MACHINE|REPLACING|REPLACE|EACH)))).*(heat pump|h/p|H\/P|h\.p\.|hp|heatpump|minisplit|mini split|mini-split|split system|ductless min|MITSUBISHI|ductless head(s)?)' ## add MITSUBISHI and remove piping ... 
###another version

# 1) Define reusable regexes
HP_TERMS = r"""
\b(
    heat\s*pumps?               |   # heat pump / heat pumps
    (d)?h[\/\.]?p[\/\.]?        |   # h/p, h.p., hp
    mini[-\s]*splits?           |   # mini split / mini-splits
    split\s*systems?            |   # split system(s)
    ductless(?:\s*mini)?        |   # ductless / ductless mini
     mitsubishi|
     VRV|VRF|
     ODU(s?)|outdoor\s*unit(s?)|
)\b
"""

LINE_TERMS = r"""
\b(
    refrig(?:erant)?\s*line(?:s|sets)? |   # refrigerant line(s)/linesets
    refer\s*lines?                      |   # 'refer lines' (common shorthand)
    piping
)\b
"""

AC_TERMS = r"\b((ductless*)?ac|air\s*condition(?:er|ing)?|a\/c)\b"

NON_HP_TERMS = r"\b(furnace|boiler|cooler|freezer)\b"

# 2) Compute helper booleans
has_hp_terms   = df_final3_summary["Description"].str.contains(HP_TERMS,   case=False, na=False, regex=True, flags=re.VERBOSE)
has_line_terms = df_final3_summary["Description"].str.contains(LINE_TERMS, case=False, na=False, regex=True, flags=re.VERBOSE)
has_ac_terms   = df_final3_summary["Description"].str.contains(AC_TERMS,   case=False, na=False, regex=True)
has_non_hp_terms = df_final3_summary["Description"].str.contains(NON_HP_TERMS, case=False, na=False, regex=True, flags=re.VERBOSE)

# 3) AC-only veto: it's AC-only if it mentions AC but NONE of the HP terms
ac_only = has_ac_terms & ~has_hp_terms
ac_only_after2015 = ac_only & (df_final3_summary["Year Completed"] >= 2015)
non_hp_terms_only = has_non_hp_terms & ~has_hp_terms
# 4) Final label:
#    - match if it has HP terms (covers "…HEAT PUMPS", "…SPLIT SYSTEMS", "MITSUBISHI", etc.)
#    - OR if it mentions line/piping terms together with HP terms somewhere in the same line
df_final3_summary["HP_related"] = (has_hp_terms | (ac_only & has_hp_terms) | (has_line_terms & has_hp_terms & ~ac_only )).astype(int)
df_final3_summary["HP_related"].value_counts() 
df_final3_summary["HP_related"] = (((has_hp_terms | (ac_only & has_hp_terms) & ~ac_only )| ac_only_after2015 ) & ~has_non_hp_terms).astype(int)
df_final3_summary["HP_related"].value_counts()
# df_final3_summary[df_final3_summary["HP_related"] == 0].to_excel('02ProcessedData/check_for_hp_related.xlsx', index=False)

C:\Users\nings\AppData\Local\Temp\ipykernel_40268\356104401.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_hp_terms   = df_final3_summary["Description"].str.contains(HP_TERMS,   case=False, na=False, regex=True, flags=re.VERBOSE)
C:\Users\nings\AppData\Local\Temp\ipykernel_40268\356104401.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_line_terms = df_final3_summary["Description"].str.contains(LINE_TERMS, case=False, na=False, regex=True, flags=re.VERBOSE)
C:\Users\nings\AppData\Local\Temp\ipykernel_40268\356104401.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_ac_terms   = df_final3_summary["Description"].str.contains(AC_TERMS,   case=False, na=False, regex=True)
C:\Users\nings\AppData\Local\Te

HP_related
1    921
Name: count, dtype: int64

HP_related
1    875
0     46
Name: count, dtype: int64

In [231]:
df_final3_summary["HP_related"].value_counts()

HP_related
1    875
0     46
Name: count, dtype: int64

## rule based conditions using tonnage inforamtion

In [232]:
import numpy as np
## Redo classification based on tonnage_cover_ratio and compressors_per_units
## if tonnage_cover_ratio >= 0.8 --> 'for whole building'
## if (Style is 'Mid-Rise' and tonnage_cover_ratio <= 0.2) | (Style is 'Low-Rise' and tonnage_cover_ratio <= 0.15) | (Style is 'Hi-Rise' and tonnage_cover_ratio <= 0.25) --> 'for common areas only'
conditions = [
    df_final3_summary['tonnage_cover_ratio'] >= 0.8,
    ((df_final3_summary['Style'] == 'Mid-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.2 ) & (df_final3_summary['tonnage_cover_ratio'] > 0)) |
    ((df_final3_summary['Style'] == 'Low-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.15) & (df_final3_summary['tonnage_cover_ratio'] > 0)) |
    ((df_final3_summary['Style'] == 'Hi-Rise') & (df_final3_summary['tonnage_cover_ratio'] <= 0.25) & (df_final3_summary['tonnage_cover_ratio'] > 0)),
     (df_final3_summary['tonnage_cover_ratio'] > 0.25),
     (df_final3_summary['tonnage_per_units'] >= 0.8),
         (df_final3_summary['tonnage_per_units'] >= 0.15) & (df_final3_summary['Style'] == 'Low-Rise')|
            (df_final3_summary['tonnage_per_units'] >= 0.2) & (df_final3_summary['Style'] == 'Mid-Rise')|
            (df_final3_summary['tonnage_per_units'] >= 0.3) & (df_final3_summary['Style'] == 'Hi-Rise'),
        (df_final3_summary['tonnage_per_units'] < 0.15) & (df_final3_summary['tonnage_per_units'] > 0), 
     (df_final3_summary['compressors_per_units'] >= 0.75),
     (df_final3_summary['compressors_per_units'] < 0.02),
    (df_final3_summary['compressors_per_units'] <= 0.75) &      (df_final3_summary['compressors_per_units'] >= 0.2),
         
]


choices = ['Whole Building', 'Common Areas Only', 'Partial Coverage','Whole Building', 'Common Areas Only', 'Partial Coverage', 'Whole Building', 'Partial Coverage', 'Common Areas Only']
df_final3_summary['coverage_classification_tonnage'] = np.select(conditions, choices, default='Uncertain')

df_final3_summary['coverage_classification_tonnage'].value_counts()
df_final3_summary['tonnage_per_units'].describe()
df_final3_summary['HP_related'].value_counts()

coverage_classification_tonnage
Uncertain            402
Partial Coverage     308
Common Areas Only    161
Whole Building        50
Name: count, dtype: int64

count    921.000000
mean       0.130526
std        0.428818
min        0.000000
25%        0.000000
50%        0.000000
75%        0.073770
max        7.800000
Name: tonnage_per_units, dtype: float64

HP_related
1    875
0     46
Name: count, dtype: int64

In [233]:

df_final3_summary[df_final3_summary['coverage_classification_tonnage']=='Uncertain'].to_csv('02ProcessedData/uncertain_coverage_classification.csv', index=False)

In [234]:

## join df_final2 with df_final3_summary on addr_clean_y to get the coverage_classification_tonnage back to df_final3

## find columns unique in df_final3_summary to merge back to df_final2
unique_cols = df_final3_summary.columns.difference(df_final2.columns)

## find columns that are shared between df_final2 and df_final3_summary
shared_cols = df_final3_summary.columns.intersection(df_final2.columns)
# unique_cols
# shared_cols
keep_columns = ['permit_id', 'tonnage_cover_ratio',
       'tonnage_total', 'tonnage_total_cumsum', 'compressors_per_units', 'tonnage_per_units',
       'compressors_final', 'compressors_final_cumsum', 'coverage_classification_tonnage','HP_related']
df_final4 = df_final2.drop(columns='HP_related').merge(df_final3_summary[keep_columns], left_on='Permit Number', right_on='permit_id', how='left')
# df_final4 = df_final4.merge(df_sum1[['addr_clean_x', 'cost_unit_ratio']], left_on='addr_clean_y', right_on='addr_clean_x', how='left')
df_final4.info()

df_final4.to_excel('02ProcessedData/final_apt_trade_with_parsed_permit_details1020_2025_with_classification.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 98 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Permit Number                    921 non-null    object 
 1   Permit_Content                   921 non-null    object 
 2   addr_clean_x                     921 non-null    object 
 3   BuildingName                     910 non-null    object 
 4   StarRating                       921 non-null    int64  
 5   Submarket                        921 non-null    object 
 6   City                             921 non-null    object 
 7   ZIPCode                          921 non-null    int64  
 8   Stories                          920 non-null    float64
 9   YearBuilt                        918 non-null    float64
 10  YearRenov                        38 non-null     float64
 11  Class                            921 non-null    object 
 12  TrueOwner             

In [235]:
df_final4['HP_related'].value_counts()

HP_related
1    875
0     46
Name: count, dtype: int64

## Rule Based Information Using Units Information

In [236]:

###- if 'Detailed Existing Land Use' containts 'mixed use|condo' & - if 'Description' contains 'lounge|single zone|lobby' --> df_final['common_area_only'] = 1

import numpy as np

df_final4['common_area_only'] = np.nan
df_final4['Number_final'] = pd.to_numeric(df_final4['Number_final'], errors='coerce')
df_final4['Units_y'] = pd.to_numeric(df_final4['Units_y'], errors='coerce')

## sort df_final4 by 'addr_clean_y' and 'Year Completed' and then do a running total of 'Number_final' by 'addr_clean_y'
df_final4 = df_final4.sort_values(by=['addr_clean_y', 'Year Completed'])
df_final4['Number_final_cumsum'] = df_final4.groupby('addr_clean_y')['Number_final'].cumsum()
df_final4['n_unit_ratio'] = df_final4['Number_final_cumsum'] / df_final4['Units_y']
# Ensure numeric columns are clean

##Note that when using np.select(conditions, choices), the first condition that evaluates to True for a given row is the one that gets applied. Later rules are ignored for that row once a match is found.

# Define conditions
conditions = [
        ## Rule 3: any cases that n_unit_ratio >=1 & not negative Number of Buildings --> for units
    (df_final4['n_unit_ratio'] >= 0.9)  ,


    ## partial coverage
    (df_final4['n_unit_ratio'] < 0.9) & (df_final4['n_unit_ratio'] >= 0.3) ,


    # Rule 2: Shared housing with very low unit ratio --> for common areas
    df_final4['Permit Class'].str.contains(
        'Commercial|Multifamily', case=False, na=False) & df_final4['is_split_system']==True &
    (df_final4['n_unit_ratio'] <= (1 / df_final4['Stories'])),



        # Rule 1: Common space keywords in shared housing --> for common areas
   df_final4['Permit Class'].str.contains(
        'Commercial|Multifamily', case=False, na=False
    ) & df_final4['Description'].str.contains(
        'lounge|single zone|lobby|retail', case=False, na=False
    ),


]




# Corresponding values
choices = ['Whole Building','Partial Coverage' ,'Common Areas Only',   'Common Areas Only']

# Apply CASE WHEN logic
df_final4['coverage_classification_units'] = np.select(conditions, choices, default= 'Uncertain')
df_final4['coverage_classification_units'].value_counts()

df_final4[['coverage_classification_tonnage','coverage_classification_units']].value_counts()

df_final4['coverage_classification_final'] = np.where(df_final4['coverage_classification_tonnage'] == 'Uncertain', df_final4['coverage_classification_units'], df_final4['coverage_classification_tonnage'])

df_final4['coverage_classification_final'] = np.where(df_final4['Description'].str.contains('each unit', case=False, na=False), "Whole Building", df_final4['coverage_classification_final'])
# df_final4['coverage_classification_final'] = np.where(df_final4['coverage_classification_units']== 'Whole Building', "Whole Building", df_final4['coverage_classification_final'])

df_final4['coverage_classification_final'].value_counts()

df_final4.groupby(['coverage_classification_tonnage','coverage_classification_units', 'coverage_classification_final']).size().head(20)
# df_final4.to_excel('03ExportedData/final_apt_trade_with_parsed_permit_details1020_2025_with_classification_v2.xlsx', index=False)
df_final4[df_final4['addr_clean_x'].str.contains('1222 E MADISON ST', case=False, na=False)]#.to_csv('02ProcessedData/check_1222E_Madison_st.csv', index=False)
df_final4.to_csv('02ProcessedData/Heat_Pump_Classification1104.csv', index=False)

coverage_classification_units
Common Areas Only    515
Uncertain            365
Whole Building        24
Partial Coverage      17
Name: count, dtype: int64

coverage_classification_tonnage  coverage_classification_units
Uncertain                        Common Areas Only                227
Partial Coverage                 Common Areas Only                172
Uncertain                        Uncertain                        170
Partial Coverage                 Uncertain                        128
Common Areas Only                Common Areas Only                 89
                                 Uncertain                         62
Whole Building                   Common Areas Only                 27
                                 Whole Building                    14
Common Areas Only                Partial Coverage                   7
Whole Building                   Uncertain                          5
Partial Coverage                 Partial Coverage                   4
                                 Whole Building                     4
Whole Building                   Partial Coverage                   4
Common Areas Only          

coverage_classification_final
Common Areas Only    388
Partial Coverage     308
Uncertain            170
Whole Building        55
Name: count, dtype: int64

coverage_classification_tonnage  coverage_classification_units  coverage_classification_final
Common Areas Only                Common Areas Only              Common Areas Only                 89
                                 Partial Coverage               Common Areas Only                  7
                                 Uncertain                      Common Areas Only                 62
                                 Whole Building                 Common Areas Only                  3
Partial Coverage                 Common Areas Only              Partial Coverage                 170
                                                                Whole Building                     2
                                 Partial Coverage               Partial Coverage                   4
                                 Uncertain                      Partial Coverage                 128
                                 Whole Building                 Partial Coverage                  

,Permit Number,Permit_Content,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,...,compressors_per_units,tonnage_per_units,compressors_final,compressors_final_cumsum,coverage_classification_tonnage,HP_related,common_area_only,Number_final_cumsum,coverage_classification_units,coverage_classification_final
73,6286852-RF,: 6286852-RF : Refrigeration Permit Record Sta...,1222 e madison st,Citizen of the Pike Pine,4,Capitol Hill,Seattle,98122,6.0,2012.0,...,0.028037,0.000000,3,3,Uncertain,1,NaN,NaN,Common Areas Only,Common Areas Only
75,6359789-RF,: 6359789-RF : Refrigeration Permit Record Sta...,1222 e madison st,Citizen of the Pike Pine,4,Capitol Hill,Seattle,98122,6.0,2012.0,...,0.037383,0.000000,1,4,Uncertain,1,NaN,NaN,Common Areas Only,Common Areas Only
76,6491890-RF,: 6491890-RF : Refrigeration Permit Record Sta...,1222 e madison st,Citizen of the Pike Pine,4,Capitol Hill,Seattle,98122,6.0,2012.0,...,0.046729,0.000000,1,5,Uncertain,1,NaN,NaN,Uncertain,Uncertain
77,6500255-RF,: 6500255-RF : Refrigeration Permit Record Sta...,1222 e madison st,Citizen of the Pike Pine,4,Capitol Hill,Seattle,98122,6.0,2012.0,...,0.056075,0.008953,1,6,Partial Coverage,1,NaN,500.0,Whole Building,Partial Coverage
74,6313657-RF,: 6313657-RF : Refrigeration Permit Record Sta...,1222 e madison st,Citizen of the Pike Pine,4,Capitol Hill,Seattle,98122,6.0,2012.0,...,0.065421,0.008953,1,7,Partial Coverage,1,NaN,NaN,Common Areas Only,Partial Coverage


In [237]:
df_final4[df_final4['coverage_classification_final']=='Uncertain'].to_csv('02ProcessedData/uncertain_coverage_classification1.csv', index=False)

In [238]:
## check which buildings have coverage_classification_tonnage == 'Whole Building' and coverage_classification_units == 'Common Areas Only'
check = df_final4[(df_final4['coverage_classification_tonnage'] == 'Whole Building') & (df_final4['coverage_classification_units'] == 'Common Areas Only')]
check = df_final4[(df_final4['coverage_classification_final'] == 'Whole Building') ]
check.to_excel('02ProcessedData/check_wholebuilding_commonareasonly_1031.xlsx', index=False)
check1 = check[['addr_clean_y', 'Description', 'tonnage_cover_ratio', 'compressors_per_units', 'coverage_classification_tonnage', 'coverage_classification_units', 'coverage_classification_final']]


In [239]:
df_final4[df_final4['BuildingName'] == 'Northaven Apartments'].head()
df_final4['HP_related'].value_counts()

## merge df_final4 with df_permit['CompletedDate','PermitNum']
df_final_final  = df_final4.drop(columns=['Completed Date']).merge(df_permit[['PermitNum', 'CompletedDate']], left_on='Permit Number', right_on='PermitNum', how='left', suffixes=('', '_y'))
# df_final_final['Year Completed'] = np.where(df_final_final['Year Completed'].isnull(), pd.to_datetime(df_final_final['Expires Date']).dt.year, df_final_final['Year Completed'])
# df_final_final['Year Completed'] =df_final_final['Year Completed_x'] 
df_final_final['Completed Date'] = df_final_final['CompletedDate']
df_final_final['Year Completed'] = np.where(
    df_final_final['Year Completed'].isnull(),
    pd.to_datetime(df_final_final['Expires Date'], errors='coerce').dt.year,
    df_final_final['Year Completed']
)
df_final_final['coverage_classification_final'].value_counts() ## 65 whole building
df_final_final['Completed Date'] = pd.to_datetime(df_final_final['Completed Date'], errors='coerce')

## keep some columns only
columns_to_keep = ['addr_clean_x','coverage_classification_final','coverage_classification_units', 'coverage_classification_tonnage', 'Completed Date', 'HP_related', 'Year Completed']
other_cols = df_apt_unique_address.columns.tolist()
df_final_final = df_final_final[columns_to_keep ].drop_duplicates().reset_index(drop=True)

## make column to upper case
df_final_final['addr_clean'] = df_final_final['addr_clean_x'].str.upper()
df_final_final['HP_covered_residential_units'] = np.where(df_final_final['coverage_classification_final'].isin(['Whole Building','Partial Coverage']), 1, 0)
# df_final_final['coverage_classification_final'] =df_final_final['coverage_classification_final'].fillna('No Permit Found')
df_final_final['coverage_classification_final'].value_counts() ## 58 whole building -- drop duplicates


## fill na of df_final_final['coverage_classification_ final'] with 'No Heat Pump Installed'
# df_final_final = df_final_final.drop_duplicates(subset=['addr_clean',]).reset_index(drop=True)

## get the quarter information like '2023Q4' from 'Completed Date'
df_final_final['Completed Quarter'] = df_final_final['Completed Date'].dt.to_period('Q').astype(str)
## merge df_final_final with df_apt_unique_address on addr_clean
df_final_final['coverage_classification_final'].value_counts()

## drop rows that have na in 'Completed Date'
df_final_final = df_final_final.dropna(subset=['Completed Date'])
df_final_final.to_excel('03ExportedData/final_apt_trade_address_classification_1029_2025.xlsx', index=False)

df_final_final['coverage_classification_final'].value_counts() ## 51 whole buidling due to drop completed date 

,Permit Number,Permit_Content,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,...,compressors_per_units,tonnage_per_units,compressors_final,compressors_final_cumsum,coverage_classification_tonnage,HP_related,common_area_only,Number_final_cumsum,coverage_classification_units,coverage_classification_final
30,6267464-RF,: 6267464-RF : Refrigeration Permit Record Sta...,11045 ne 8th ave,Northaven Apartments,2,Pinehurst,Seattle,98125,8.0,1972.0,...,0.015544,0.0,3,3,Partial Coverage,1,NaN,3.0,Common Areas Only,Partial Coverage


HP_related
1    875
0     46
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    388
Partial Coverage     308
Uncertain            170
Whole Building        55
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    378
Partial Coverage     301
Uncertain            162
Whole Building        51
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    378
Partial Coverage     301
Uncertain            162
Whole Building        51
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    345
Partial Coverage     257
Uncertain            152
Whole Building        44
Name: count, dtype: int64

In [240]:
df_final_final['coverage_classification_final'].value_counts()
### check how many buildings have 'No Permit Found' but HP_related == 1


coverage_classification_final
Common Areas Only    345
Partial Coverage     257
Uncertain            152
Whole Building        44
Name: count, dtype: int64

In [241]:

df_ht = df_apt_unique_address.merge(df_final_final[['addr_clean_x', 'coverage_classification_final', 'coverage_classification_units', 'coverage_classification_tonnage', 'HP_related', 'Year Completed']], left_on='addr_clean', right_on='addr_clean_x', how='left')
df_ht['coverage_classification_final'].value_counts()
## fill na in coverage_classification_final as 'No Permit Found'
## fill na in HP_related as 0
df_ht['HP_related'] = df_ht['HP_related'].fillna(0)

# df_ht['HP_covered_residential_units'] = np.where(df_ht['coverage_classification_final'].isin(['Whole Building','Partial Coverage']), 1, 0)
df_ht = df_ht.drop_duplicates(subset=['addr_clean']).reset_index(drop=True)
df_ht.info()
df_ht.to_excel('03ExportedData/final_apt_trade_address_classification_1024_2025.xlsx', index=False)


coverage_classification_final
Common Areas Only    345
Partial Coverage     257
Uncertain            153
Whole Building        44
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   addr_clean                       943 non-null    object 
 1   BuildingName                     917 non-null    object 
 2   StarRating                       943 non-null    int64  
 3   Submarket                        943 non-null    object 
 4   City                             943 non-null    object 
 5   ZIPCode                          943 non-null    int64  
 6   Stories                          941 non-null    float64
 7   YearBuilt                        931 non-null    float64
 8   YearRenov                        64 non-null     float64
 9   Class                            943 non-null    object 
 10  TrueOwner                        918 non-null    object 
 11  Units                            942 non-null    float64
 12  latitude              

## Time Series Data Generated


In [242]:
import pandas as pd
import openpyxl as ox
import numpy as np
import re
import os
os.getcwd()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'f:\\01PHD\\016Heat_Pump_Research\\01SN_Rproject'

In [329]:
import pandas as pd

data_apt_time = pd.read_excel('01Data/Seattle_HP_Aptmt Units_Timeseries.xlsx')
data_apt_time.shape

(47905, 49)

In [ ]:
df_permit = pd.read_csv('01Data/Trade_Permits_20251003.csv')


In [323]:
df_final_final = pd.read_excel('03ExportedData/final_apt_trade_address_classification_1029_2025.xlsx')
df_final_final_conf = pd.read_csv('03ExportedData/predictions_with_confidence.csv')
df_final_final_conf.describe() 



,StarRating,ZIPCode,Stories,YearBuilt,YearRenov,Units,latitude,longitude,LEED,ParkingSpaces,...,tonnage_total,tonnage_total_cumsum,compressors_per_units,tonnage_per_units,compressors_final,compressors_final_cumsum,HP_related,common_area_only,Number_final_cumsum,confidence
count,921.000000,921.000000,920.000000,918.000000,38.000000,921.000000,921.000000,921.000000,139.0,485.000000,...,921.000000,921.000000,921.000000,921.000000,921.000000,921.000000,921.000000,0.0,4.750000e+02,921.000000
mean,3.838219,98113.756786,7.702174,2007.759259,2010.500000,151.511401,47.625924,-122.331468,1.0,143.633320,...,10.411585,18.290106,0.116538,0.130526,6.737242,14.346363,0.950054,NaN,1.299925e+04,0.929839
std,0.518038,11.484182,6.491176,23.104612,9.802785,120.208488,0.040586,0.026169,0.0,132.737038,...,62.629744,73.436142,0.244440,0.428818,24.249752,36.388820,0.217951,NaN,1.994122e+05,0.152709
min,2.000000,98101.000000,1.000000,1900.000000,1978.000000,15.000000,47.520120,-122.388860,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000e+00,0.483236
25%,4.000000,98104.000000,5.000000,2009.000000,2006.000000,60.000000,47.609927,-122.347320,1.0,41.000000,...,0.000000,0.000000,0.024070,0.000000,1.000000,2.000000,1.000000,NaN,2.000000e+00,1.000000
50%,4.000000,98112.000000,6.000000,2015.000000,2015.000000,108.000000,47.618765,-122.327610,1.0,112.000000,...,0.000000,0.000000,0.046122,0.000000,2.000000,5.000000,1.000000,NaN,5.000000e+00,1.000000
75%,4.000000,98122.000000,7.000000,2019.000000,2016.000000,234.000000,47.656521,-122.315500,1.0,200.000000,...,4.000000,8.000000,0.088889,0.073770,4.000000,11.000000,1.000000,NaN,1.000000e+01,1.000000
max,5.000000,98199.000000,44.000000,2023.000000,2019.000000,761.000000,47.732554,-122.261650,1.0,916.000000,...,1490.000000,1490.000000,2.472222,7.800000,329.000000,335.000000,1.000000,NaN,3.076440e+06,1.000000


In [292]:
df_final_final.columns
df_final_final_conf.columns

df_final_final['addr_clean'].nunique()

## get the completed time from df_permit to df_final_final_conf
df_final_final_conf = df_final_final_conf.drop(columns= 'Completed Date').merge(df_permit[['PermitNum', 'CompletedDate']], left_on='Permit Number', right_on='PermitNum', how='left')
## rename CompletedDate to Completed Date
df_final_final_conf = df_final_final_conf.rename(columns={'CompletedDate': 'Completed Date'})



Index(['addr_clean_x', 'coverage_classification_final',
       'coverage_classification_units', 'coverage_classification_tonnage',
       'Completed Date', 'HP_related', 'Year Completed', 'addr_clean',
       'HP_covered_residential_units', 'Completed Quarter'],
      dtype='object')

Index(['Permit Number', 'Permit_Content', 'addr_clean_x', 'BuildingName',
       'StarRating', 'Submarket', 'City', 'ZIPCode', 'Stories', 'YearBuilt',
       ...
       'compressors_final_cumsum', 'coverage_classification_tonnage',
       'HP_related', 'common_area_only', 'Number_final_cumsum',
       'coverage_classification_units', 'coverage_classification_final',
       'source', 'predicted_label', 'confidence'],
      dtype='object', length=105)

471

In [293]:
df_final_final_conf.head()


,Permit Number,Permit_Content,addr_clean_x,BuildingName,StarRating,Submarket,City,ZIPCode,Stories,YearBuilt,...,HP_related,common_area_only,Number_final_cumsum,coverage_classification_units,coverage_classification_final,source,predicted_label,confidence,PermitNum,Completed Date
0,6691699-RF,: 6691699-RF : Refrigeration Permit Record Sta...,100 denny way,Alexan 100,4,Lower Queen Anne,Seattle,98109,7.0,2019.0,...,1,NaN,90.0,Partial Coverage,Partial Coverage,labeled,Partial Coverage,1.0,6691699-RF,12/2/2019
1,6908623-RF,: 6908623-RF : Refrigeration Permit Record Sta...,100 denny way,Alexan 100,4,Lower Queen Anne,Seattle,98109,7.0,2019.0,...,1,NaN,NaN,Common Areas Only,Common Areas Only,labeled,Common Areas Only,1.0,6908623-RF,3/27/2023
2,7052479-RF,: 7052479-RF : Refrigeration Permit Record Sta...,100 denny way,Alexan 100,4,Lower Queen Anne,Seattle,98109,7.0,2019.0,...,1,NaN,91.0,Uncertain,Common Areas Only,labeled,Common Areas Only,1.0,7052479-RF,NaN
3,6172962-RF,: 6172962-RF : Refrigeration Permit Record Sta...,100 taylor ave n,Taylor 28,4,South Lake Union,Seattle,98109,6.0,2008.0,...,1,NaN,3.0,Uncertain,Partial Coverage,labeled,Partial Coverage,1.0,6172962-RF,7/22/2008
4,6515780-RF,: 6515780-RF : Refrigeration Permit Record Sta...,1000 8th ave,The Mill at First Hill,3,First Hill,Seattle,98104,14.0,1949.0,...,1,NaN,1.0,Common Areas Only,Partial Coverage,labeled,Partial Coverage,1.0,6515780-RF,4/7/2017


In [294]:

df_final_final_conf['addr_clean_x'].nunique()
## make Completed Quarter in df_final_final_conf from 'Completed Date'
df_final_final_conf['Completed Date'] = pd.to_datetime(df_final_final_conf['Completed Date'], errors='coerce')
df_final_final_conf['Completed Quarter'] = df_final_final_conf['Completed Date'].dt.to_period('Q').astype(str)  

## sort df_final_final by completed quarter 
df_final_final_conf = df_final_final_conf.sort_values(by='Completed Quarter')

# drop some duplication columns in df_final_final_conf if in the same addr_clean and Completed Quarter
# df_final_final_conf = df_final_final_conf[~df_final_final_conf.set_index(['addr_clean_x', 'Completed Quarter']).index.duplicated(keep='last')]
df_final_final_conf.info()

## keep only those Completed Quarter is not null
df_final_final_conf = df_final_final_conf.dropna(subset=['Completed Date'])
df_final_final_conf.info()



492

<class 'pandas.core.frame.DataFrame'>
Index: 921 entries, 766 to 279
Columns: 107 entries, Permit Number to Completed Quarter
dtypes: bool(4), datetime64[ns](1), float64(38), int64(8), object(56)
memory usage: 751.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 817 entries, 766 to 187
Columns: 107 entries, Permit Number to Completed Quarter
dtypes: bool(4), datetime64[ns](1), float64(38), int64(8), object(56)
memory usage: 667.0+ KB


In [ ]:
## merge df_final_final with df_final_final_conf on addr_clean and Completed Quarter to get predicted_label and confidence_score
# df_final_final = df_final_final.merge(df_final_final_conf[['addr_clean_x', 'Completed Quarter', 'predicted_label']], left_on=['addr_clean_x', 'Completed Quarter'], right_on=['addr_clean_x', 'Completed Quarter'], how='left')
# df_final_final.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   addr_clean_x                     798 non-null    object        
 1   coverage_classification_final    798 non-null    object        
 2   coverage_classification_units    798 non-null    object        
 3   coverage_classification_tonnage  798 non-null    object        
 4   Completed Date                   798 non-null    datetime64[ns]
 5   HP_related                       798 non-null    int64         
 6   Year Completed                   798 non-null    int64         
 7   addr_clean                       798 non-null    object        
 8   HP_covered_residential_units     798 non-null    int64         
 9   Completed Quarter                798 non-null    object        
 10  predicted_label                  158 non-null    object       

In [301]:
## keep only those with Completed Date not null or empty
df_final_final_conf = df_final_final_conf.dropna(subset=['Completed Date'])
df_final_final_conf = df_final_final_conf[df_final_final_conf['Completed Date'] != '']
df_final_final_conf.describe()
# df_final_final['predicted_label'].value_counts()
df_final_final_conf['predicted_label'].value_counts()
df_final_final_conf['coverage_classification_final'].value_counts()

,StarRating,ZIPCode,Stories,YearBuilt,YearRenov,Units,latitude,longitude,LEED,ParkingSpaces,...,tonnage_total_cumsum,compressors_per_units,tonnage_per_units,compressors_final,compressors_final_cumsum,HP_related,common_area_only,Number_final_cumsum,confidence,Completed Date
count,817.000000,817.000000,816.000000,814.000000,29.000000,817.000000,817.000000,817.000000,122.0,424.000000,...,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,0.0,4.310000e+02,817.000000,817
mean,3.834761,98113.713586,7.474265,2008.351351,2013.517241,147.310894,47.626158,-122.331092,1.0,143.835283,...,16.866573,0.113408,0.122301,6.667075,13.903305,0.953488,NaN,7.182571e+03,0.926506,2016-10-30 11:00:57.282741760
min,2.000000,98101.000000,1.000000,1900.000000,1991.000000,15.000000,47.520120,-122.388860,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000e+00,0.483236,2003-02-04 00:00:00
25%,4.000000,98104.000000,5.000000,2009.000000,2014.000000,60.000000,47.609726,-122.347320,1.0,46.000000,...,0.000000,0.023256,0.000000,1.000000,2.000000,1.000000,NaN,2.000000e+00,1.000000,2014-01-17 00:00:00
50%,4.000000,98112.000000,6.000000,2015.000000,2016.000000,107.000000,47.619133,-122.327040,1.0,113.000000,...,0.000000,0.044444,0.000000,2.000000,5.000000,1.000000,NaN,5.000000e+00,1.000000,2017-06-02 00:00:00
75%,4.000000,98121.000000,7.000000,2019.000000,2017.000000,211.000000,47.657786,-122.314640,1.0,200.000000,...,5.000000,0.086538,0.060000,5.000000,11.000000,1.000000,NaN,1.000000e+01,1.000000,2020-06-22 00:00:00
max,5.000000,98199.000000,44.000000,2023.000000,2019.000000,761.000000,47.732554,-122.261650,1.0,916.000000,...,1490.000000,2.361111,7.800000,329.000000,335.000000,1.000000,NaN,3.076437e+06,1.000000,2025-09-29 00:00:00
std,0.512937,11.213994,6.161772,22.301675,6.511642,113.546511,0.041285,0.026352,0.0,133.173401,...,74.826794,0.234065,0.436300,23.636790,36.168440,0.210719,NaN,1.481848e+05,0.155681,NaN


predicted_label
Common Areas Only    464
Partial Coverage     306
Whole Building        47
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    350
Partial Coverage     262
Uncertain            158
Whole Building        47
Name: count, dtype: int64

In [304]:

df= data_apt_time
df.info()
df_final_final = df_final_final_conf
## sort df by Completed Date
df_final_final['Completed Date'] = pd.to_datetime(df_final_final['Completed Date'], errors='coerce')
df_final_final = df_final_final.sort_values(by='Completed Date')

df_final_final['Completed Quarter'] = df_final_final['Completed Date'].dt.to_period('Q').astype(str)

## join df and df_final_final on addr_clean and for 'Quarter_clean' >= 'Completed Quarter'
df_final_final.columns
df_final_final['predicted_label'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47905 entries, 0 to 47904
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Number                 47905 non-null  object 
 1   Quarter                47905 non-null  object 
 2   Status                 47905 non-null  object 
 3   AskingRentPerUnit      42015 non-null  float64
 4   AskingRentPerSF        41455 non-null  float64
 5   AskingRentGrowthYr     41157 non-null  float64
 6   EffectiveRentPerUnit   42015 non-null  float64
 7   EffectiveRentPerSF     41455 non-null  float64
 8   EffectiveRentGrowthYr  41157 non-null  float64
 9   VacancyUnits           47828 non-null  float64
 10  VacancyTotal           47828 non-null  float64
 11  Address                47905 non-null  object 
 12  BuildingName           46519 non-null  object 
 13  StarRating             47905 non-null  int64  
 14  ES                     2450 non-null   float64
 15  Es

Index(['Permit Number', 'Permit_Content', 'addr_clean_x', 'BuildingName',
       'StarRating', 'Submarket', 'City', 'ZIPCode', 'Stories', 'YearBuilt',
       ...
       'common_area_only', 'Number_final_cumsum',
       'coverage_classification_units', 'coverage_classification_final',
       'source', 'predicted_label', 'confidence', 'PermitNum',
       'Completed Date', 'Completed Quarter'],
      dtype='object', length=107)

predicted_label
Common Areas Only    464
Partial Coverage     306
Whole Building        47
Name: count, dtype: int64

In [305]:

# Work on a copy of the original dataset
# Clean and standardize the 'Quarter' column
df['Quarter_clean'] = (
    df['Quarter']
    .str.replace('QTD', '', regex=False)
    .str.strip()
    .str.replace(' ', '', regex=False)
)

# Convert cleaned quarter strings to quarterly periods
df['Quarter_period'] = pd.PeriodIndex(df['Quarter_clean'], freq='Q')

# Convert to timestamp (start of quarter) and then shift to quarter end
df['Quarter_end'] = df['Quarter_period'].dt.to_timestamp() + pd.offsets.QuarterEnd(0)



In [306]:
df['addr_clean']

0                       3800 AURORA AVE N
1                  4821 FAUNTLEROY WAY SW
2                  10735 ROOSEVELT WAY NE
3                   12111215 DEXTER AVE N
4                          1307 N 43RD ST
                       ...               
47900                     705 S WELLER ST
47901    7923 MARTIN LUTHER KING JR WAY S
47902                5216 BROOKLYN AVE NE
47903                      526 19TH AVE E
47904                       1818 20TH AVE
Name: addr_clean, Length: 47905, dtype: object

In [307]:
df_final_final['addr_clean'] = df_final_final['addr_clean_y'].str.upper()


In [308]:
df_final_final = df_final_final.drop(columns=['Units_y', 'Stories_y'], errors='ignore') #'addr_clean_x', 'addr_clean_y',
df = df.merge(df_final_final, left_on=['addr_clean', 'Quarter_clean'], right_on=['addr_clean', 'Completed Quarter'], how='left')
# df.to_excel('02ProcessedData/final_apt_time_series_with_HP_classification_1031_2025_ML_predictions.xlsx', index=False)
# df = df_final_final.merge(df, left_on=['addr_clean', 'Completed Quarter'], right_on=['addr_clean', 'Quarter_clean'],  how='full')


In [309]:
df['predicted_label'].value_counts()
df['coverage_classification_final'].value_counts()
df['coverage_classification_tonnage'].value_counts()

predicted_label
Common Areas Only    342
Partial Coverage     217
Whole Building        32
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    259
Partial Coverage     190
Uncertain            110
Whole Building        32
Name: count, dtype: int64

coverage_classification_tonnage
Uncertain            277
Partial Coverage     188
Common Areas Only     97
Whole Building        29
Name: count, dtype: int64

In [310]:
# df.info()
## sort the df by addr_clean and Quarter_end and Completed Date
df = df.sort_values(by=['addr_clean', 'Quarter_end', 'Completed Quarter'])

## for each addr_clean, do a forward fill of Completed Date (only for blank Completed Date)
df['Completed Quarter'] = df.groupby('addr_clean')['Completed Date'].fillna(method='ffill')


## for each addr_clean, do a forward fill of coverage_classification_final where Quarter_end >= Completed Date (only for blank coverage_classification_final)
df['coverage_classification_final'] = df.groupby('addr_clean')['coverage_classification_final'].fillna(method='ffill')
df['predicted_label'] = df.groupby('addr_clean')['predicted_label'].fillna(method='ffill')


C:\Users\nings\AppData\Local\Temp\ipykernel_40268\4058730891.py:6: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Completed Quarter'] = df.groupby('addr_clean')['Completed Date'].fillna(method='ffill')
C:\Users\nings\AppData\Local\Temp\ipykernel_40268\4058730891.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Completed Quarter'] = df.groupby('addr_clean')['Completed Date'].fillna(method='ffill')
C:\Users\nings\AppData\Local\Temp\ipykernel_40268\4058730891.py:10: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['coverage_cl

In [311]:
df['predicted_label'].value_counts()
df['coverage_classification_final'].value_counts()

predicted_label
Common Areas Only    6582
Partial Coverage     3780
Whole Building        242
Name: count, dtype: int64

coverage_classification_final
Common Areas Only    4531
Partial Coverage     3173
Uncertain            2658
Whole Building        242
Name: count, dtype: int64

## forward filling for any given coverage information


In [312]:
import pandas as pd

# Ensure quarters are Periods for comparison
df_final_final['Completed Quarter'] = pd.PeriodIndex(df_final_final['Completed Quarter'], freq='Q')
df['Quarter_clean'] = pd.PeriodIndex(df['Quarter_clean'], freq='Q')

# Prepare event table (rename for clarity)
events = (df_final_final
          .sort_values(['addr_clean', 'Completed Quarter'])
          .drop_duplicates(['addr_clean', 'Completed Quarter'], keep='last')
          .rename(columns={'Completed Quarter': 'Quarter_clean'}))

# Outer merge all quarters + events
timeline = pd.concat([
    df[['addr_clean', 'Quarter_clean', 'coverage_classification_final','predicted_label']],
    events[['addr_clean', 'Quarter_clean']] #, 'coverage_classification_final'
]).sort_values(['addr_clean', 'Quarter_clean','coverage_classification_final','predicted_label']).drop_duplicates(['addr_clean', 'Quarter_clean','coverage_classification_final','predicted_label'], keep='last')

# Forward-fill coverage within each address
timeline['coverage_filled'] = timeline.groupby('addr_clean')['coverage_classification_final'].ffill()
timeline['predicted_label'] = timeline.groupby('addr_clean')['predicted_label'].ffill()
# Keep only df's original rows, and fill NAs
result = (df.drop(columns=['predicted_label']).merge(timeline[['addr_clean', 'Quarter_clean', 'coverage_filled', 'predicted_label']],
                 on=['addr_clean', 'Quarter_clean'], how='left'))
# result['coverage_classification_final'] = result['coverage_classification_final'].fillna(result['coverage_filled'])


In [313]:
result['coverage_filled'].value_counts()
result['predicted_label'].value_counts()

coverage_filled
Common Areas Only    4789
Partial Coverage     3375
Uncertain            2794
Whole Building        280
Name: count, dtype: int64

predicted_label
Common Areas Only    6947
Partial Coverage     4011
Whole Building        280
Name: count, dtype: int64

In [314]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48600 entries, 0 to 48599
Columns: 158 entries, Number_x to predicted_label
dtypes: datetime64[ns](3), float64(71), int64(3), object(79), period[Q-DEC](2)
memory usage: 58.6+ MB


In [315]:
df= result.copy()
df.describe(include = 'object').T


,count,unique,top,freq
Number_x,48600,945,MBSEA566,107
Quarter,48600,96,2023 Q3,962
Status,48600,1,Existing,48600
Address_x,48600,943,900 University St,107
BuildingName_x,47207,917,Madison Court,193
...,...,...,...,...
coverage_classification_final,11238,4,Common Areas Only,4808
source,1225,2,labeled,993
PermitNum,1225,591,6491890-RF,3
coverage_filled,11238,4,Common Areas Only,4789


In [325]:

import numpy as np
# Compare timestamps
df['is_before'] = df['Completed Quarter'] < df['Quarter_end']
df['is_after'] = df['Completed Quarter'] > df['Quarter_end']
df['is_same_or_after'] = df['Completed Quarter'] >= df['Quarter_end']

df['HP_related'] = df.groupby('addr_clean')['HP_related'].ffill()

df['HP_installed'] = 0
df.loc[(df['Completed Quarter'].notna()) & (df['HP_related'] == 1) & (df['is_before']), 'HP_installed'] = 1
df['HP_installed'].value_counts()
# df[df['HP_installed'] == 1][['Address', 'addr_clean', 'Quarter', 'Completed Date', 'Completed Quarter', 'Quarter_end', 'is_before', 'HP_related', 'HP_installed']].head(20)

df[df['HP_installed'] == 1]['addr_clean'].value_counts()
df['coverage_filled'] = df['coverage_filled'].fillna('No Heat Pump Installed')
## for 'predicted_label' if coverage_filled is not 'Uncertain', use coverage_filled to fill predicted_label
df['predicted_label'] = np.where(df['coverage_filled'] != 'Uncertain', df['coverage_filled'], df['predicted_label'])

df['predicted_label'] = df['predicted_label'].fillna('No Heat Pump Installed')

df['HP_covered_residential_units'] = np.where(df['predicted_label'].isin(['Whole Building','Partial Coverage']), 1, 0)
df[df['HP_covered_residential_units'] == 1]['addr_clean'].nunique()

df['HP_covered_residential_units'].value_counts()






HP_installed
0    37838
1    10762
Name: count, dtype: int64

addr_clean
522 N 85TH ST         86
1942 WESTLAKE AVE     85
2201 4TH AVE          85
3310 E SPRING ST      84
1701 N 45TH ST        83
                      ..
2101 S GRAND ST        2
320 N 85TH ST          2
301 MINOR AVE N        2
609 W NICKERSON ST     2
1700 17TH AVE          2
Name: count, Length: 352, dtype: int64

183

HP_covered_residential_units
0    44309
1     4291
Name: count, dtype: int64

In [317]:
df.groupby('addr_clean')['HP_related'].value_counts()
a = df.groupby('addr_clean')['HP_related'].nunique()
a.value_counts().sort_index(ascending=False)
a[a>1]


addr_clean          HP_related
100 DENNY WAY       1.0           19
100 TAYLOR AVE N    1.0           63
1000 8TH AVE        1.0           31
1001 MINOR AVE      1.0           19
1008 N 109TH ST     1.0           22
                                  ..
912 DEXTER AVE N    0.0            6
913 S JACKSON ST    1.0           27
9176 HOLMAN RD NW   1.0           28
9400 RAINIER AVE S  1.0           10
975 JOHN ST         1.0           44
Name: count, Length: 378, dtype: int64

HP_related
2     25
1    328
0    590
Name: count, dtype: int64

addr_clean
1208 PINE ST              2
1260 REPUBLICAN ST        2
1315 E JEFFERSON ST       2
1414 10TH AVE             2
1420 E MADISON ST         2
1630 43RD AVE E           2
2030 8TH AVE              2
2121 6TH AVE              2
2301 E UNION ST           2
301 MINOR AVE N           2
320 E PINE ST             2
3272 FUHRMAN AVE E        2
3920 STONE WAY N          2
4020 NE 55TH ST           2
4730 CALIFORNIA AVE SW    2
504 5TH AVE S             2
507 NE NORTHGATE WAY      2
523 BROADWAY E            2
550 BROADWAY              2
601 E PIKE ST             2
630 BOREN AVE N           2
6600 ROOSEVELT WAY NE     2
743 N 35TH ST             2
900 UNIVERSITY ST         2
912 DEXTER AVE N          2
Name: HP_related, dtype: int64

In [318]:
df.head()

,Number_x,Quarter,Status,AskingRentPerUnit,AskingRentPerSF,AskingRentGrowthYr,EffectiveRentPerUnit,EffectiveRentPerSF,EffectiveRentGrowthYr,VacancyUnits,...,PermitNum,Completed Date,Completed Quarter,coverage_filled,predicted_label,is_before,is_after,is_same_or_after,HP_installed,HP_covered_residential_units
0,MASEA210,2019 Q2,Existing,2537.6290,2.870085,-0.006353,2388.7258,2.701834,-0.064611,39.2134,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
1,MASEA210,2019 Q3,Existing,2452.6613,2.773771,-0.044884,2373.8387,2.684635,-0.075509,11.9994,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
2,MASEA210,2019 Q4,Existing,2470.6613,2.793810,-0.041690,2470.5323,2.793678,-0.041728,7.7772,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
3,MASEA210,2020 Q1,Existing,2505.4516,2.832279,-0.033379,2475.5323,2.798470,-0.044886,2.7220,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
4,MASEA210,2020 Q2,Existing,2542.4194,2.873854,0.001888,2293.3710,2.593208,-0.039919,2.7364,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0


In [319]:
df.head(10)
df['coverage_filled'].value_counts()
df['HP_related'] = df.groupby('addr_clean')['HP_related'].ffill()
df['HP_related'] = df['HP_related'].fillna(0).astype(int)
df.to_excel('03ExportedData/time_series_with_hp_updated1105.xlsx', index=False)


,Number_x,Quarter,Status,AskingRentPerUnit,AskingRentPerSF,AskingRentGrowthYr,EffectiveRentPerUnit,EffectiveRentPerSF,EffectiveRentGrowthYr,VacancyUnits,...,PermitNum,Completed Date,Completed Quarter,coverage_filled,predicted_label,is_before,is_after,is_same_or_after,HP_installed,HP_covered_residential_units
0,MASEA210,2019 Q2,Existing,2537.6290,2.870085,-0.006353,2388.7258,2.701834,-0.064611,39.2134,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
1,MASEA210,2019 Q3,Existing,2452.6613,2.773771,-0.044884,2373.8387,2.684635,-0.075509,11.9994,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
2,MASEA210,2019 Q4,Existing,2470.6613,2.793810,-0.041690,2470.5323,2.793678,-0.041728,7.7772,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
3,MASEA210,2020 Q1,Existing,2505.4516,2.832279,-0.033379,2475.5323,2.798470,-0.044886,2.7220,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
4,MASEA210,2020 Q2,Existing,2542.4194,2.873854,0.001888,2293.3710,2.593208,-0.039919,2.7364,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
5,MASEA210,2020 Q3,Existing,2491.6129,2.817259,0.015881,2472.0484,2.795155,0.041372,3.9258,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
6,MASEA210,2020 Q4,Existing,2448.0323,2.768467,-0.009159,2141.9194,2.422305,-0.133013,7.1250,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
7,MASEA210,2021 Q1,Existing,2413.3065,2.729695,-0.036778,2111.5161,2.388344,-0.147046,6.0003,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
8,MASEA210,2021 Q2,Existing,2609.7258,2.951852,0.026473,2595.8226,2.936132,0.131881,2.2862,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0
9,MASEA210,2021 Q3,Existing,2685.9516,3.038185,0.077997,2679.0806,3.030438,0.083749,0.0000,...,NaN,NaT,NaT,No Heat Pump Installed,No Heat Pump Installed,False,False,False,0,0


coverage_filled
No Heat Pump Installed    37362
Common Areas Only          4789
Partial Coverage           3375
Uncertain                  2794
Whole Building              280
Name: count, dtype: int64

In [320]:
df['HP_related'].value_counts()
df['coverage_filled'].value_counts()



HP_related
0    37830
1    10770
Name: count, dtype: int64

coverage_filled
No Heat Pump Installed    37362
Common Areas Only          4789
Partial Coverage           3375
Uncertain                  2794
Whole Building              280
Name: count, dtype: int64

In [326]:
## check if how many addresses have more than 3 types of coverage_filled

df.groupby('addr_clean')['coverage_filled'].value_counts()

## get the how many unique addressses in each type of coverage_filled and drop duplicates using the last record

## check if how many addresses have more than 3 types of final_coverage_classification
df_final_final.groupby('addr_clean')['coverage_classification_final'].nunique().sort_values(ascending=False )

df_sorted = df.sort_values(by=['addr_clean','Completed Quarter'])  # optional if order matters
r= df_sorted.drop_duplicates(subset=['addr_clean'], keep='last')
counts = r['predicted_label'].value_counts()
counts



addr_clean             coverage_filled       
10 CLAY ST             No Heat Pump Installed    19
100 DENNY WAY          Partial Coverage          14
                       Common Areas Only          5
100 TAYLOR AVE N       Partial Coverage          63
                       No Heat Pump Installed     1
                                                 ..
9425 35TH AVE NE       No Heat Pump Installed    96
954 E UNION ST         No Heat Pump Installed    38
9720 ROOSEVELT WAY NE  No Heat Pump Installed    96
975 JOHN ST            Common Areas Only         44
999 HIAWATHA PL SOUTH  No Heat Pump Installed    40
Name: count, Length: 1217, dtype: int64

addr_clean
900 UNIVERSITY ST         3
888 WESTERN AVE           3
523 BROADWAY E            3
5240 UNIVERSITY WAY NE    3
6404 9TH AVE NE           3
                         ..
110 10TH AVE S            1
1167 REPUBLICAN ST        1
1170 REPUBLICAN ST        1
120 10TH AVE E            1
120 6TH AVE S             1
Name: coverage_classification_final, Length: 471, dtype: int64

predicted_label
No Heat Pump Installed    778
Common Areas Only          96
Partial Coverage           55
Whole Building             14
Name: count, dtype: int64

In [322]:
df['predicted_label'].value_counts()

predicted_label
No Heat Pump Installed    37362
Common Areas Only          6947
Partial Coverage           4011
Whole Building              280
Name: count, dtype: int64

In [328]:
df.shape
df['addr_clean'].nunique()

(48600, 163)

943